In [71]:
from pytube import YouTube
import os
import requests
from pydub import AudioSegment
import openai
from typing import Final
from dotenv import load_dotenv

# STEP 0: load our token from somwhere safe
load_dotenv()
OPEN_API_TOKEN: Final[str] = os.getenv('OPEN_API_TOKEN')
openai.api_key = OPEN_API_TOKEN


In [3]:
link = "https://www.youtube.com/watch?v=IHQr0HCIN2w"

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'age_restricted',
 'author',
 'bypass_age_gate',
 'caption_tracks',
 'captions',
 'channel_id',
 'channel_url',
 'check_availability',
 'description',
 'embed_html',
 'fmt_streams',
 'from_id',
 'initial_data',
 'js',
 'js_url',
 'keywords',
 'length',
 'metadata',
 'publish_date',
 'rating',
 'register_on_complete_callback',
 'register_on_progress_callback',
 'streaming_data',
 'streams',
 'thumbnail_url',
 'title',
 'vid_info',
 'views',
 'watch_html']

In [31]:
def get_mp4_from_youtube_link(link):
    youtube_object = YouTube(link)
    print(youtube_object)
    youtube_object.title
    youtube_object.thumbnail_url
    #yt.streams.filter(file_extension='mp4')
    #save_video_path = "G:/temp/youtube-videos/"
    output_file_name = youtube_object.title + ".mp4"
    #download in local file system
    youtube_object.streams.filter(progressive=True, file_extension='mp4').order_by('resolution').desc().first().download(filename = output_file_name)
    return output_file_name
    

#! consider temp file https://www.geeksforgeeks.org/python-tempfile-module/

In [64]:
#get video
output_file_name = get_mp4_from_youtube_link(link)

def does_file_exist(output_file_name):
    return os.path.isfile(output_file_name)


<pytube.__main__.YouTube object: videoId=IHQr0HCIN2w>


In [77]:
def get_audio_from_youtube_link(link):
    youtube_object = YouTube(link)
    print(youtube_object)
    youtube_object.title
    youtube_object.thumbnail_url
    #download in local file system
    output_file_name = youtube_object.title +"_audio" + "." + "mp4"
    youtube_object.streams.filter(only_audio=True).first().download(filename = output_file_name) 
    #youtube_object.streams.filter(progressive=True, file_extension=extension).order_by('resolution').desc().first().download(filename = output_file_name)
    return output_file_name

#! consider temp file https://www.geeksforgeeks.org/python-tempfile-module/

In [78]:
output_file_name = get_audio_from_youtube_link(link)
does_file_exist(output_file_name)

<pytube.__main__.YouTube object: videoId=IHQr0HCIN2w>


True

In [79]:
def segment_and_calculate_volume(audio_file, segment_length_ms):
    # Load the audio file
    audio = AudioSegment.from_file(audio_file)

    # Calculate the total number of segments
    num_segments = len(audio) // segment_length_ms

    # Initialize a list to store the speech volumes of each segment
    segment_volumes = []

    # Iterate over the audio and segment it
    for i in range(num_segments):
        start_time = i * segment_length_ms
        end_time = (i + 1) * segment_length_ms
        segment = audio[start_time:end_time]

        # Extract just the speech portion (assuming it's in a certain range of frequencies)
        speech_segment = segment.low_pass_filter(3000)

        # Calculate the average volume level in decibels (dB)
        speech_volume = speech_segment.dBFS

        segment_volumes.append(speech_volume)

    return segment_volumes


In [80]:

def calculate_average_speech_volume(audio_file):
    "average audio volume"
    # Load the audio file
    audio = AudioSegment.from_file(audio_file)

    # Extract just the speech portion (assuming it's in a certain range of frequencies)
    speech = audio.low_pass_filter(3000)

    # Calculate the average volume level in decibels (dB)
    speech_volume = speech.dBFS

    return speech_volume


In [81]:
audio_file = output_file_name #path to the audio file
avg_speech_volume = calculate_average_speech_volume(audio_file)
print("Avg file Speech volume (dB):", avg_speech_volume)

segment_length_ms = 5000  # Length of each segment in milliseconds (adjust as needed)
segment_volumes = segment_and_calculate_volume(audio_file, segment_length_ms)
print("Speech volumes of each segment (dB):", segment_volumes)

#median


Avg file Speech volume (dB): -19.289763653230835
Speech volumes of each segment (dB): [-18.50554206726807, -18.87016599770511, -17.733048987860155, -20.45658425833052, -23.215307620099786, -21.85063908384111, -21.778773471902856, -18.121379814180216, -21.643204995293843, -23.238367517639116, -26.798762431925404, -19.674312513663352, -22.75195031905927, -21.873630675055878, -20.807202631414224, -24.517219648342422, -20.476162829642632, -21.573028494558283, -19.08935758746965, -19.600584338070888, -22.079971856435606, -23.56380790418884, -20.775263840281465, -20.501400299130776, -20.442626534552325, -20.348304225453816, -20.353824333448998, -23.10470643328731, -23.512023038441615, -16.711359856937108, -17.46564601109545, -21.3101789237276, -15.912432974107663, -18.22663857535366, -20.27686025081777, -33.01877707802652, -22.150987896341654, -19.373211666569198, -24.811992298861057, -37.053842065562876, -12.209281077207091, -9.68244554397174, -47.89752007507736, -32.389504052013066, -21.93

In [87]:


audio_file = open(output_file_name, "rb")

transcript = openai.audio.transcriptions.create(
  file=audio_file,
  model="whisper-1",
  response_format="verbose_json",
  timestamp_granularities=["segment"]
)



In [86]:
print(transcript.segments)


[{'id': 0, 'seek': 0, 'start': 0.0, 'end': 7.0, 'text': ' Freeze! End of the line, Cortez!', 'tokens': [50364, 48096, 0, 6967, 295, 264, 1622, 11, 28522, 4371, 0, 50714], 'temperature': 0.0, 'avg_logprob': -0.3803545832633972, 'compression_ratio': 1.2627737522125244, 'no_speech_prob': 0.3153893053531647}, {'id': 1, 'seek': 0, 'start': 7.0, 'end': 8.0, 'text': ' Alright.', 'tokens': [50714, 2798, 13, 50764], 'temperature': 0.0, 'avg_logprob': -0.3803545832633972, 'compression_ratio': 1.2627737522125244, 'no_speech_prob': 0.3153893053531647}, {'id': 2, 'seek': 0, 'start': 13.0, 'end': 14.0, 'text': ' Here we go.', 'tokens': [51014, 1692, 321, 352, 13, 51064], 'temperature': 0.0, 'avg_logprob': -0.3803545832633972, 'compression_ratio': 1.2627737522125244, 'no_speech_prob': 0.3153893053531647}, {'id': 3, 'seek': 0, 'start': 17.0, 'end': 21.0, 'text': ' Well, well, well, boys. Looks like we got a Mexican standoff.', 'tokens': [51214, 1042, 11, 731, 11, 731, 11, 6347, 13, 10027, 411, 321, 65

In [ ]:
#some code for emotion from hume
# url = "https://api.hume.ai/v0/batch/jobs"

# payload = "{\"models\":{\"face\":{\"fps_pred\":3,\"prob_threshold\":0.99,\"identify_faces\":false,\"min_face_size\":60,\"save_faces\":false},\"prosody\":{\"granularity\":\"utterance\",\"window\":{\"length\":4,\"step\":1}},\"language\":{\"granularity\":\"word\"}},\"transcription\":{\"language\":null,\"identify_speakers\":false,\"confidence_threshold\":0.5},\"notify\":false}"
# headers = {
#     "accept": "application/json; charset=utf-8",
#     "content-type": "application/json; charset=utf-8",
#     "X-Hume-Api-Key": "YOUR API KEY HERE"
# }

# response = requests.post(url, data=payload, headers=headers)

# print(response.text)

True


1. get youtube mp4
2. get subtitles - enhanced input
3. convert youtube to anime
4. maybe black and white it
5. get screenshots every 5 seconds + subtitle for timestamp
6. put subtitles onto screenshot, in speechbubble? to the right person? (can hard code it, like stage it - switch sides whenever person speaking)
7. format into comic panels
8. compile publish as pdf

video - popular